# Multi-Class Prediction of Obesity Risk

**Programa de Engenharia de Sistemas e Computação**

**CPS833 - Data Mining**

**Professor**: Geraldo Zimbrão da Silva

**Aluno**: Luiz Henrique Souza Caldas

## Importação de bibliotecas

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from yellowbrick.target import FeatureCorrelation
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, cross_val_score

# Pré-processamento

In [2]:
def data_overview(tr_d, te_d):
    # Identifica duplicatas no conjunto de dados de treinamento
    tr_duplicates = tr_d.duplicated().sum()
    # Identifica duplicatas no conjunto de dados de teste
    te_duplicates = te_d.duplicated().sum()

    # Impressão dos resultados
    print(f"Number of duplicate rows in training data: {tr_duplicates}")
    print(f"Number of duplicate rows in test data: {te_duplicates}")

    # Se existirem duplicatas, exibe mais detalhes
    if tr_duplicates > 0:
        print("Duplicate rows in training data:")
        display(tr_d[tr_d.duplicated(keep=False)])  # Mostra todas as linhas duplicadas

    if te_duplicates > 0:
        print("Duplicate rows in test data:")
        display(te_d[te_d.duplicated(keep=False)])  # Mostra todas as linhas duplicadas

In [66]:
# Importação dos dados
train_dataset = pd.read_csv('train.csv')

# Verificação de dados ausentes (limpeza de dados)
print(f"{train_dataset.isnull().any().sum()} dados faltantes")

# Verificação de linhas duplicadas (redução de dimensionalidade)
print(f"{train_dataset.duplicated().sum()} linhas duplicadas")
data_overview(train_dataset,train_dataset)

# TODO Verificação de ouliers (redução de dimensionalidade)


# Codificando features categóricas com Label Encoder (transformação de dados)
label_encoder = LabelEncoder()
train_dataset_encoded = train_dataset.copy()  # Faz uma cópia do dataset para evitar alterações no original
for col in train_dataset.columns:
    if train_dataset[col].dtype == 'object':  # Verifica se a coluna é categórica
        train_dataset_encoded[col] = label_encoder.fit_transform(train_dataset[col])

# Verificando a covariância entre as features e o label (redução de dimensionalidade)
# grafico = FeatureCorrelation(labels=train_dataset_encoded.columns[1:17])
# grafico.fit(train_dataset_encoded.iloc[:,1:17].values, train_dataset_encoded.iloc[:,17].values)
# grafico.show();
# train_dataset_preprocessed = train_dataset_encoded[['CALC','CH2O','CAEC','family_history_with_overweight','Weight','Age','NObeyesdad']]

# Separação entre features e labels
features = train_dataset_encoded.iloc[:,1:17].values # features removendo o id
labels = train_dataset_encoded.iloc[:,17].values # labels
# features = train_dataset_preprocessed.iloc[:,1:6].values # features removendo o id
# labels = train_dataset_preprocessed.iloc[:,6].values # labels

# Escalonando os dados (transformação de dados)
scaler = StandardScaler()
features = scaler.fit_transform(features)

0 dados faltantes
0 linhas duplicadas
Number of duplicate rows in training data: 0
Number of duplicate rows in test data: 0


## Escolha da técnica

In [67]:
resultados_naive_bayes = []
resultados_logistica = []
resultados_forest = []

for i in range(30):
    x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size = 0.2, stratify = labels, random_state=i)
    
    naive_bayes = GaussianNB() # criação do modelo Naive Bayes
    naive_bayes.fit(x_train,y_train) # treinamento do modelo Naive Bayes
    resultados_naive_bayes.append(accuracy_score(y_test, naive_bayes.predict(x_test))) # avaliação do modelo Naive Bayes
    
    logistica = LogisticRegression(max_iter=1000) # criação do modelo de Regressão Logística
    logistica.fit(x_train,y_train) # treinamento do modelo de Regressão Logística
    resultados_logistica.append(accuracy_score(y_test, logistica.predict(x_test))) # avaliação do modelo de Regressão Logística

    random_forest = RandomForestClassifier() # criação do modelo de Random Forest
    random_forest.fit(x_train,y_train) # treinamento do modelo de Random Forest
    resultados_forest.append(accuracy_score(y_test, random_forest.predict(x_test))) # avaliação do modelo de Random Forest

In [68]:
medias = [np.mean(resultados_naive_bayes), np.mean(resultados_logistica),np.mean(resultados_forest)]
std = [np.std(resultados_naive_bayes), np.std(resultados_logistica),np.std(resultados_forest)]
print(f"Naive Bayes com média {medias[0]} e desvio padrão {std[0]}")
print(f"Regressão Logística com média {medias[1]} e desvio padrão {std[1]}")
print(f"Random Forest com média {medias[2]} e desvio padrão {std[2]}")

Naive Bayes com média 0.6625561978163135 e desvio padrão 0.005871924080901997
Regressão Logística com média 0.8582931920359668 e desvio padrão 0.0038558134761785923
Random Forest com média 0.8998554913294796 e desvio padrão 0.0038432142632271347


## Otimização dos hiperparâmetros